In [1]:
import numpy as np
import random
# import trimesh
from mpl_toolkits import mplot3d
from matplotlib import pyplot
import copy
import os
import matplotlib.patches as mpatches
import sys
import time
import statistics



In [2]:

def initial_cell(state,Size):
    if Size == 2:
        state['CellType'].append('SC')
        state['Volume'].append(1.82)

    elif Size == 1:
        state['CellType'].append('bc')
        state['Volume'].append(0.5571)
    return state

def cell(state,Size):
    if Size == 2:
        rand_cell = np.random.randint(0,3)#Random cell type
        if rand_cell == 0: #transition
            state['CellType'].append('TR')
            state['Volume'].append(3.02)
        elif rand_cell == 1: #body centered
            state['CellType'].append('SC')
            state['Volume'].append(1.82)
        elif rand_cell == 2:  #simple cubic with rounded edges
            state['CellType'].append('FC')
            state['Volume'].append(3.69)
    elif Size == 1:
        rand_cell = np.random.randint(0,3)#Random cell type
        if rand_cell == 0:  #simple cubic with rounded edges
            state['CellType'].append('sol')
            state['Volume'].append(1)
        elif rand_cell == 1:  #simple cubic with rounded edges
            state['CellType'].append('sc')
            state['Volume'].append(0.7839)
        elif rand_cell == 2:  #simple cubic with rounded edges
            state['CellType'].append('bc')
            state['Volume'].append(0.5571)
    return state

In [3]:
def constraint(State,V_max): #Maximum volume constraint
    VOL = np.array(State['Volume']).reshape(-1,1)
    CELL =  np.array(State['CellType']).reshape(-1,1)
    POS = np.array(State['Position'])
    cell = CELL.flatten()
    V_sup = sum(VOL)
    if V_sup > V_max:
        ver = False
#         print('Volume Violation:',V_sup,'>',V_max)
    else:
        ver = True
    return V_sup,ver

In [4]:
def initialize(Unit,Sup,init): ## Initialize structure
    T_state = {'CellType':[],'Volume':[],'Position':[],'Size':[]}
    for i in range(len(Unit)):
        new_cell = {'CellType':[],'Volume':[]}
        if i<len(Sup):
            size = 2
        else:
            size = 1
        new_cell = initial_cell(new_cell,size)

#         new_cell = cell(new_cell,size)
        T_state['CellType'].append(new_cell['CellType'][0])
        T_state['Volume'].append(new_cell['Volume'][0])
        T_state['Position'].append(Unit[i].tolist())
        T_state['Size'].append(size)
    Vol,ver = constraint(T_state,0)#,0)
    return Vol,T_state

def evaluate(State,init,DisMax,StrMax):   #Evaluate Objectives and constraints
    num_cells=len(State['CellType'])
    CELL = np.array(State['CellType']).reshape(-1,1)
    POS = np.array(State['Position'])*1e-3
    Size = np.array(State['Size'])
    h =1e-3
    Dis, Stress, E = FEM_Thermal(CELL,POS,Size,h,init,DisMax,StrMax)   
#     Dis = 1
#     Stress = 1
#     E = 1
    return E,init,Dis,Stress

In [5]:
def metopolis(E_prev,Vol,Vol2, new_state,prev_state,T,init,Dis,Str,DisMax,StrMax): #boolean of True/False
    E1 = E_prev
    Vol1= Vol
    Dis1 = Dis
    Str1 = Str
    E2,init,Dis2,Str2= evaluate(new_state,init,DisMax,StrMax)
    if E1 < E2: # if previous objective is better
        P_acc = np.exp(-(E2-E1)/T) #minimize
        comp = np.random.uniform(0, 1) #random probability
        if P_acc > comp: # Probability is better than rand number
            accept = True
            E = E2 #Objective
            Vol = Vol2
            Dis = Dis2
            Str = Str2
        else:
            accept = False
            P_acc = 0
            E = E1
            Vol = Vol1
            Dis = Dis1
            Str = Str1
    elif E2 == False:
        accept = False
        P_acc = 0
        E = E1
        Vol = Vol1
        Dis = Dis1
        Str = Str1
    else: # if current objective is better
        P_acc = 1
        accept = True
        E = E2
        Vol = Vol2
        Dis = Dis2
        Str = Str2
    return accept,E, Vol, P_acc,Dis, Str

In [6]:
def ShapeAnneal(it,state,T,limit,n,alpha,Sup,Unit,E,Vol,init,V_max,Dis,Str,DisMax,StrMax):
    T_0 = T #Initial temperature
    P_acc = 1 #Probability of acceptance
    Res_loss = [] #initialize objective list for storage at each iteration
    V_iter = [] #initialize volume list for storage at each iteration
    Temp = [] #initialize temperature list for storage at each iteration
    P_acc_iter = [] #initialize probability of acceptance list for storage at each iteration
    Str_iter = [] #initialize p-Norm stress list for storage at each iteration
    Dis_iter = [] #initialize displacement list for storage at each iteration
    while it < 800:
        success = 0 
        for i in range(n):
            P_acc_iter.append(P_acc) #store P_acc
            Res_loss.append(E)
            V_iter.append(Vol)
            Str_iter.append(Str)
            Dis_iter.append(Dis)
            prev_state = copy.deepcopy(state)
            Total=len(Unit)
            Int = T_0/2
            Tun = 1
            if i == 0:
                print('New Objective:',E)
            if T<=T_0 and T > Int:
                RANGE = int(Total/400)
            elif T>Tun and T<= Int:
                RANGE = int(Total/400)
            else:
                RANGE = int(Total/400)
            new_state = copy.deepcopy(state)
            for I in range (RANGE):
                Size = np.random.randint(1,3)
                if Size == 2:
                    rand_cell_select = np.random.randint(0,len(Sup)) #randomly select 2mm unit cell
                else:
                    rand_cell_select = np.random.randint(len(Sup),len(Unit)-len(Sup)) #randomly select 1mm unit cell
                new_cell = {'CellType':[],'Volume':[]} #initialize new cell
                new_cell = cell(new_cell,Size) #obtain new random cell
                new_state['CellType'][rand_cell_select]=new_cell['CellType'][0]
                new_state['Volume'][rand_cell_select]=new_cell['Volume'][0]
            Vol2,ver2 = constraint(new_state,V_max)
            if ver2 == True:
                test,E_curr,Vol_curr, P_acc,Dis_curr, Str_curr = metopolis(E,Vol,Vol2,new_state,prev_state,T,init,Dis,Str,DisMax,StrMax)
                E = E_curr
                Vol = Vol_curr
                Dis = Dis_curr
                Str = Str_curr
#                     print('Objective',E)
            else:
                test = False
            it+=1 
            if test == True:
                state = new_state
                success += 1
            else:
                state = prev_state

            if success > limit:
                print('~~~~~~~~New Temp~~~~~~~~~~')
                break
        if success == 0:
#             print('Success = 0')
            break
        T = T*alpha
    print('Final Objective',E,'Final Volume',np.round(Vol,3), 'Final Displacement', np.round(Dis,3), 'Final Stress', Str)
    print('Final State',state)
    return state,Res_loss,V_iter,P_acc_iter,Str_iter,Dis_iter


In [12]:
##### 
start_time = time.time()
T = 50 #initial temperature
n = 100 #number of mutations

limit = n/2 #maximum number of limits
alpha = 0.5 #cooling coefficient
np.set_printoptions(threshold=sys.maxsize)
CellCount = []
SOLU = []
start = 0
end = 30
for RUN in range(start,end):
    start_time = time.time()
    titleS = f"{os.getcwd()}\\AsymSupport.txt"
    Sup = np.loadtxt(titleS)
    titleS2 = f"{os.getcwd()}\\AsymSingle.txt"
    Sing = np.loadtxt(titleS2)
    Unit=np.concatenate((Sup,Sing))

    %run ./AA_Adapter_MAPDL.ipynb #call pyAnsys code to evaluate objective function and constraints
    mapdl.exit() #close if existing mapdl is open (If running for firs time, comment out this line)
    path = os.getcwd()
    mapdl = launch_mapdl(nproc = 8, run_location=path,override = True, license_type = "ansys") #start MAPDL
    mapdl.clear() #Clear any existing data     
    
    DisMax = 40 #Maximum displacement
    StrMax = 0.66 #Maximum stress
    V_max = 3808 #Maximum volume
    init = 0 #Indicator that this is the first time it is running
    Vol,state = initialize(Unit,Sup,init)
    print('state',state) #initial configuration
    E,init,Dis,Str= evaluate(state,init,DisMax,StrMax)
    init = 1 #Indicator that the iteration loop has begun
    print('Initial Objective',E,'Volume',np.round(Vol,3),'Max Displacement',np.round(Dis,3),'Max Stress',Str)
    CELL = np.array(state['CellType'])
    sc = np.count_nonzero(CELL == 'SC') #count number of SC
    tr = np.count_nonzero(CELL == 'TR')
    fc = np.count_nonzero(CELL == 'FC')
    sc1 = np.count_nonzero(CELL == 'sc')
    bv1 = np.count_nonzero(CELL == 'bc')
    sol = np.count_nonzero(CELL == 'sol')
    Cell_count = [sc,tr,fc,sc1,bv1,sol]
    CellCount.append(Cell_count)
    print('Simple Cubic:', sc,'Transition Cubic:', tr,'Face-Centered Vertile Cubic:',fc)
    print('SC1:', sc1,'BV1 Cubic:', bv1,'Solid Cubic:',sol)
    it = 1 #iteration number is 1
    solutions,R_loss,V_iter,P_acc_iter,Str_iter,Dis_iter= ShapeAnneal(it,state,T,limit,n,alpha,Sup,Unit,E,Vol,init,V_max,Dis,Str,DisMax,StrMax)
    mapdl.exit() #close out of mapdl
    CELL = np.array(solutions['CellType'])
    SOLU.append(solutions) #store final state
    sc = np.count_nonzero(CELL == 'SC')
    tr = np.count_nonzero(CELL == 'TR')
    fc = np.count_nonzero(CELL == 'FC')
    sc1 = np.count_nonzero(CELL == 'sc')
    bv1 = np.count_nonzero(CELL == 'bc')
    sol = np.count_nonzero(CELL == 'sol')
    Cell_count = [sc,tr,fc,sc1,bv1,sol]
    CellCount.append(Cell_count)
    print('Simple Cubic:', sc,'Transition Cubic:', tr,'Face-Centered Vertile Cubic:',fc)
    print('SC1:', sc1,'BV1 Cubic:', bv1,'Solid Cubic:',sol)
    print("--- %s seconds ---" % (time.time() - start_time)) #run time   
    Iter_data = np.concatenate((np.array(R_loss).reshape(-1,1),np.round(np.array(V_iter).reshape(-1,1),3),np.round(np.array(Str_iter).reshape(-1,1),3),np.round(np.array(Dis_iter).reshape(-1,1),3)),axis = 1)
    with open('ASymm50_TSA_%i' %RUN + '.txt', 'w') as f: #export iteration results
        content = str(Iter_data)
        f.write(content)
        f.close()
CC = np.array(CellCount)
with open('CellCount_ASymm50_TSA_%i'%start+'-%i'%end+'.txt', 'w') as f: #export cell count
    content = str(CC)
    f.write(content)
    f.close()
with open('SOLU_ASymm50_TSA_%i'%start+'-%i'%end+'.txt', 'w') as f: #export sample results
    content = str(SOLU)
    f.write(content)
    f.close()

state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'S

Structural boundary conditions attached--- 1.1029999256134033 seconds ---
Initial Objective [-1564.55120478] Volume [2576.267] Max Displacement 29.354 Max Stress 0.08829556029043847
Simple Cubic: 631 Transition Cubic: 0 Face-Centered Vertile Cubic: 0
SC1: 0 BV1 Cubic: 2563 Solid Cubic: 0
New Objective: [-1564.55120478]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-1734.21912261]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-1898.00833705]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2089.30644888]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2202.56960119]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2335.93573761]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2490.13470194]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2625.02537144]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2766.58286753]
New Objective: [-2937.03335249]
New Objective: [-3053.00617258]
New Objective: [-3165.84543219]
Final Objective [-3260.47974397] Final Volume [3642.345] Final Displacement 17.049 Final Stress 0.0821610128359

Simple Cubic: 152 Transition Cubic: 194 Face-Centered Vertile Cubic: 285
SC1: 463 BV1 Cubic: 1659 Solid Cubic: 441
--- 49901.8413476944 seconds ---
state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC'

Structural boundary conditions attached--- 1.1779992580413818 seconds ---
Initial Objective [-1564.55120478] Volume [2576.267] Max Displacement 29.354 Max Stress 0.08829556029043847
Simple Cubic: 631 Transition Cubic: 0 Face-Centered Vertile Cubic: 0
SC1: 0 BV1 Cubic: 2563 Solid Cubic: 0
New Objective: [-1564.55120478]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-1770.32832707]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-1911.99935452]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2092.2879595]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2244.70685228]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2350.27905437]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2432.14686321]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2614.97894855]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2745.93784701]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2871.34582675]
New Objective: [-3015.46113885]
Final Objective [-3164.43561371] Final Volume [3581.551] Final Displacement 17.212 Final Stress 0.08599212826395931
F

Simple Cubic: 174 Transition Cubic: 178 Face-Centered Vertile Cubic: 279
SC1: 415 BV1 Cubic: 1751 Solid Cubic: 397
--- 45866.426112651825 seconds ---
state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'S

...

Simple Cubic: 149 Transition Cubic: 178 Face-Centered Vertile Cubic: 304
SC1: 407 BV1 Cubic: 1717 Solid Cubic: 439
--- 47527.92215895653 seconds ---
state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC

Structural boundary conditions attached--- 1.118997573852539 seconds ---
Initial Objective [-1564.55120478] Volume [2576.267] Max Displacement 29.354 Max Stress 0.08829556029043847
Simple Cubic: 631 Transition Cubic: 0 Face-Centered Vertile Cubic: 0
SC1: 0 BV1 Cubic: 2563 Solid Cubic: 0
New Objective: [-1564.55120478]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-1780.65986047]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-1949.72663053]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2158.69941937]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2332.11642697]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2445.8269962]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2545.78008994]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2759.27487935]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2925.39984898]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3057.16258396]
New Objective: [-3156.70598679]
New Objective: [-3248.93044443]
Final Objective [-3358.72142048] Final Volume [3634.343] Final Displacement 17.296 Fin

Simple Cubic: 163 Transition Cubic: 178 Face-Centered Vertile Cubic: 290
SC1: 436 BV1 Cubic: 1668 Solid Cubic: 459
--- 52041.35724616051 seconds ---
state {'CellType': ['SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC

Structural boundary conditions attached--- 1.1050004959106445 seconds ---
Initial Objective [-1564.55120478] Volume [2576.267] Max Displacement 29.354 Max Stress 0.08829556029043847
Simple Cubic: 631 Transition Cubic: 0 Face-Centered Vertile Cubic: 0
SC1: 0 BV1 Cubic: 2563 Solid Cubic: 0
New Objective: [-1564.55120478]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-1783.55973878]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2005.21853755]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2215.07285713]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2364.36475758]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2463.5265505]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2626.70283144]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2792.56487424]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-2931.64199107]
~~~~~~~~New Temp~~~~~~~~~~
New Objective: [-3012.54250643]
New Objective: [-3144.09356512]
New Objective: [-3239.82427189]
Final Objective [-3320.89427603] Final Volume [3646.58] Final Displacement 16.644 Fin

Simple Cubic: 155 Transition Cubic: 184 Face-Centered Vertile Cubic: 292
SC1: 430 BV1 Cubic: 1668 Solid Cubic: 465
--- 52123.08496236801 seconds ---
